In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import wordcloud as wcl
import tweepy as tw
import seaborn as sns

In [6]:
data = yf.download('^VIX', start='2025-03-10', end='2025-04-10')
vix_data = data[['Open', 'Close']]
print(vix_data)

[*********************100%***********************]  1 of 1 completed

Price            Open      Close
Ticker           ^VIX       ^VIX
Date                            
2025-03-10  24.700001  27.860001
2025-03-11  27.940001  26.920000
2025-03-12  26.879999  24.230000
2025-03-13  24.920000  24.660000
2025-03-14  24.350000  21.770000
2025-03-17  22.889999  20.510000
2025-03-18  20.830000  21.700001
2025-03-19  21.840000  19.900000
2025-03-20  19.520000  19.799999
2025-03-21  20.020000  19.280001
2025-03-24  19.129999  17.480000
2025-03-25  17.580000  17.150000
2025-03-26  17.230000  18.330000
2025-03-27  18.360001  18.690001
2025-03-28  19.080000  21.650000
2025-03-31  24.110001  22.280001
2025-04-01  22.059999  21.770000
2025-04-02  22.299999  21.510000
2025-04-03  26.379999  30.020000
2025-04-04  30.120001  45.310001
2025-04-07  60.130001  46.980000
2025-04-08  44.040001  52.330002
2025-04-09  50.980000  33.619999
